<h2>Lab 2: LU Factorization</h2>
<b>Demo Date: </b> Sept. 22 <br>
<b>Due Date: </b> Sept. 25

In this lab you will implement two versions of the LU Factorization algorithm: the one presented in the pseudocode of the textbook and another that uses Numpy operations with matrices. We will then compare the performance of the two implementations on artificial problems. Here we will assume that the linear system has a single solution and that pivoting isn't needed (we will study pivoting in our Tuesday lecture).

In class we discussed how the matrix $A$ of a linear system $Ax = b$ can be decomposed into a lower triangular matrix $L$ and an upper triangular matrix $U$. i.e., $A = LU$. The decomposition allows us to write the original system as $LUx = b$. Then, we make $y = Ux$ and solve the system $Ly = b$ with an algorithm called forward-substitution. The solution $y$ is then be used to discover the solution to the original problem, by making $Ux = y$ and solving this system with the back-substitution algorithm. 

In class we studied the back-substitution algorithm, which is very similar to the forward-substitution algorithm. Back-substitution solves systems whose matrix A is an upper triangular matrix, while forward-substitution solves systems whose matrix A is a lower triangular matrix. 

Before moving forward, please take a look at the pseudocode of the forward and back-substitution algorithms in the textbook (see Algorithm 2.1 on page 64 and Algorithm 2.2 on page 65). If you understand the forward and back-substitution algorithms, then please go ahead and study the pseudocode of the LU-factorization (see Algorithm 2.3 on page 68 of the textbook). 

Let's now implement these three algorithms to solve the system used as example in class. 

\begin{align*}
Ax = \begin{bmatrix}
1 & 2 & 2 \\
4 & 4 & 2 \\
4 & 6 & 4 \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\
x_2 \\
x_3 \\
\end{bmatrix} = 
\begin{bmatrix}
3 \\
6 \\
10 \\
\end{bmatrix} = b
\end{align*}

In [1]:
import numpy as np
import time
import copy
import scipy.linalg

A = np.array([[1, 2, 2], [4, 4, 2], [4, 6, 4]])
b = np.array([3, 6, 10]).reshape(3, 1)

Finish the implementation of the algorithms below. The implementation of these algorithms should follow the pseudocode of the textbook. 

The output should be $x = [-1, 3, -1]^T$

In [2]:
def forward_substituion(L, b):
    x = np.zeros((b.shape[0]))
    for j in range(0, L.shape[1]):
        # singular matrix
        if L[j][j] == 0:
            break 
        x[j] = b[j]/ L[j][j]
    
        for i in range(j, L.shape[0]):
            b[i] = b[i] - L[i][j] * x[j]
    return x

def back_substituion(U, b):
    x = np.zeros((b.shape[0]))
    for j in range(U.shape[1]-1,-1,-1):
        # singular matrix
        if U[j][j] == 0:
            break 
        x[j] = b[j]/ U[j][j]
    
        for i in range(0,j):
            b[i] = b[i] - U[i][j] * x[j]
    return x.transpose()

def lu_factor_v1(A):
    L = np.eye(A.shape[0])
    
    for k in range(A.shape[1]):
        if A[k][k] == 0:
            break
        for i in range(k+1,A.shape[0]):
            L[i][k] = A[i][k]/A[k][k]
            #A[i][k] = 0
        for j in range(k+1,A.shape[0]):
            for i in range(k+1,A.shape[0]):
                A[i][j] = A[i][j] - L[i][k]*A[k][j]
                
    #print(L)
    #print(A)
    return L, A

n = len(b)
A1 = copy.deepcopy(A)
b1 = copy.deepcopy(b)

L, U = lu_factor_v1(A1)
#print("L:\n", L)
#print("U:\n", U)
y = forward_substituion(L, b1)
#print(y)
x = back_substituion(U, y)

print('x: ', x)

x:  [-1.  3. -1.]


Next, we will write a vectorized implementation of the LU factorization. For that you will modify your previous implementation. The only for-loop you will keep in the vectorized implementation is the outer loop of the non-vectorized implementation, the one that iterates over the $k-1$ columns of $A$. You should rely on numpy functions to rewrite the code.

In [3]:
def lu_factor_v2(A):
    L = np.eye(A.shape[0])
    U = A.copy()
    for k in range(U.shape[1]):
        if U[k][k] == 0:
            break
       # for i in range(k+1,A.shape[0]):
       #     M[i][k] = A[i][k]/A[k][k]
       #     A[i][k] = 0
       # for j in range(k+1,A.shape[1]):
       #     for i in range(k+1,A.shape[1]):
       #         A[i][j] = A[i][j] - M[i][k]*A[k][j]
        
        temp = U[k+1:,k]/U[k,k]
        L[k+1:,k] = temp
        # adding the dimention to increase the shape of the temp, to make temp*U[k] caculable
        # here is the citation for adding dimention：
        # https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html
        temp = np.expand_dims(temp, axis=1)
        U[k+1:] = U[k+1:] - np.multiply(temp,U[k])
        # print(U[k])
        # print(U[k+1:, k]/U[k, k] * U[k])
        # print(U[k+1:, k]/U[k, k])
    #print(U)
    #print(L)    
        
    return L, U
    
L, U = lu_factor_v2(copy.deepcopy(A))
y = forward_substituion(L, copy.deepcopy(b))
x = back_substituion(U, y)
print('x: ', x)

x:  [-1.  3. -1.]


In the following snippet we will compare the running time of the vectorized and non-vectorized implementation by performing the LU-factorization on larger $200 \times 200$ matrices. 

In [4]:
running_time_vectorized = []
running_time_non_vectorized = []

for _ in range(10):
    test_A = np.tril(np.random.rand(200, 200))
    
    A = copy.deepcopy(test_A)
    start = time.time()
    L, U = lu_factor_v1(A)
    end = time.time()
    running_time_non_vectorized.append(end - start)
    
    A = copy.deepcopy(test_A)
    start = time.time()
    L, U = lu_factor_v2(A)
    end = time.time()
    running_time_vectorized.append(end - start)

print('Non-Vectorized: %.4f seconds' % np.average(running_time_non_vectorized))
print('Vectorized: %.4f seconds' % np.average(running_time_vectorized))

Non-Vectorized: 5.2373 seconds
Vectorized: 0.0225 seconds
